In [1]:
import sys
import os
import time
import shutil
from typing import Literal


run: str = str(int(time.time()))
is_colab: bool = 'google.colab' in sys.modules

if is_colab:
  !pip install poetry~=1.2
  !poetry config virtualenvs.create false
  %cd /content/
  if not os.path.exists('Unlearn-Saliency'):
    !git clone https://github.com/LeonardoSanBenitez/Unlearn-Saliency.git
  %cd /content/Unlearn-Saliency/PEM_composition_text
  !sed -i 's/python = "~3.10"/python = "~3.11"/' ./pyproject.toml
  !poetry install --no-root --no-interaction --no-ansi

  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
else:
  # TODO: ensure we are in the right folder?
  pass


# Set environment variables
#os.environ["WANDB_PROJECT"] = "civil.adapter"
#os.environ["WANDB_WATCH"] = "false"
os.environ["WANDB_DISABLED"] = "true"  # TODO: enable some model monitoring

os.environ["HF_HOME"] = "./assets/models/hf"  # Same as TRANSFORMERS_CACHE?
os.environ["TRANSFORMERS_CACHE"] = "./assets/models/hf"
os.environ["HF_DATASETS_CACHE"] = "./assets/datasets"
os.environ["HF_METRICS_CACHE"] = "./assets/metrics"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5"  # TODO: check actual number of GPUs

# Fine-Tuning the Full Model (FFT)
Fine-tunes the entire `gpt2-large` model on the `civil_comments` dataset without using adapters.
- Full fine-tuning (FFT) updates all model parameters.
- Uses a lower learning rate (`1e-5`) and more gradient accumulation (`16` steps).
- Computationally expensive but results in a fully fine-tuned model.

In [ ]:
# Set training parameters
save_dir = f"./assets/models/full_finetuning/{run}"

# Run training
!python run_clm_noconcat.py \
    --model_name_or_path gpt2-large \
    --dataset_name "civil_comments" \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --save_total_limit 5 \
    --learning_rate 1e-5 \
    --overwrite_output_dir \
    --fp16 \
    --do_train \
    --do_eval \
    --eval_steps 200 \
    --save_steps 200 \
    --evaluation_strategy "steps" \
    --save_strategy "steps" \
    --num_train_epochs 5 \
    --output_dir {save_dir} \
    --warmup_steps 0 \
    --warmup_ratio 0.06 \
    --gradient_accumulation_steps 16 \
    --weight_decay 0.01 \
    --report_to "wandb"

In [ ]:
if is_colab:
    assert os.path.isdir(os.path.join("/content/Unlearn-Saliency/PEM_composition_text/", save_dir))
    shutil.copytree(
        os.path.join("/content/Unlearn-Saliency/PEM_composition_text/", save_dir),
        os.path.join("/content/gdrive/MyDrive/temp/unlearning/PEM_composition_text/", save_dir)
    )

# Training just an Adapter
Instead of fine-tuning the full model, we train an IA3 adapter on top of gpt2-large.

* Only adapter parameters are updated, keeping the base model frozen.
* Uses a higher learning rate (5e-3) and fewer gradient accumulation steps (8).
* More memory-efficient than full fine-tuning.

In [2]:
# Set training parameters
adapter_config: Literal["lora", "ia3"] = "lora"
save_dir = f"./assets/models/{adapter_config}/{run}"
# TODO: save training params somewhere in the filesystem

# Run adapter training
!poetry run ./run_clm_noconcat.py \
    --model_name_or_path "gpt2-medium" \
    --dataset_name "civil_comments" \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 8 \
    --num_train_epochs 1 \
    --save_total_limit 5 \
    --learning_rate 5e-3 \
    --warmup_steps 0 \
    --warmup_ratio 0.06 \
    --fp16 \
    --do_train \
    --do_eval \
    --overwrite_output_dir \
    --output_dir {save_dir} \
    --train_adapter \
    --adapter_config {adapter_config}

Skipping virtualenv creation, as specified in config file.
Traceback (most recent call last):
  File "/src/./run_clm_noconcat.py", line 659, in <module>
    main()
  File "/src/./run_clm_noconcat.py", line 218, in main
    model_args, data_args, training_args, adapter_args = parser.parse_args_into_dataclasses()
  File "/src/transformers/hf_argparser.py", line 224, in parse_args_into_dataclasses
    obj = dtype(**inputs)
  File "<string>", line 103, in __init__
  File "/src/transformers/training_args.py", line 1091, in __post_init__
    raise ValueError(
ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA devices.


In [ ]:
if is_colab:
    assert os.path.isdir(os.path.join("/content/Unlearn-Saliency/PEM_composition_text/", save_dir))
    shutil.copytree(
        os.path.join("/content/Unlearn-Saliency/PEM_composition_text/", save_dir),
        os.path.join("/content/gdrive/MyDrive/temp/unlearning/PEM_composition_text/", save_dir)
    )

# Evaluation & Scoring
Generates text samples from a trained adapter and evaluates them.

Uses scaling (PEM composition) to adjust the adapter’s influence (scale=0.7).
Saves generated samples and evaluates them using a separate prediction script.

In [ ]:
# Set output directories
adapter_config: Literal["lora", "ia3"] = "lora"
scale = 0.7  # TODO: save this as metadata somewhere in the filesystem
trained_adapter_dir = f"./assets/models/{adapter_config}/{run}"
trained_adapter_negated_dir = f"./assets/models/{adapter_config}_negated/{run}"
outputs_dir = f"./assets/outputs/{run}"

save_gen_file = os.path.join(outputs_dir, "gen.txt")
save_pred_file = os.path.join(outputs_dir, "pred.csv")

# Generate text samples with the trained adapter
!poetry run ./gpt2_scale.py \
    --model_type "gpt2" \
    --model_name_or_path "gpt2-medium" \
    --prompt "I don't care if this is controversial" \
    --fp16 \
    --num 1000 \
    --temperature 1.0 \
    --length 128 \
    --adapter_config {adapter_config} \
    --load_adapter {trained_adapter_dir} \
    --save_dir {save_gen_file} \
    --model_save_dir={trained_adapter_negated_dir} \
    --scale={scale}

!poetry run toxicity_score.py \
    --model_name "unbiased" \
    --input {save_gen_file} \
    --save_to {save_pred_file}

In [ ]:
if is_colab:
    assert os.path.isdir(os.path.join("/content/Unlearn-Saliency/PEM_composition_text/", outputs_dir))
    shutil.copytree(
        os.path.join("/content/Unlearn-Saliency/PEM_composition_text/", outputs_dir),
        os.path.join("/content/gdrive/MyDrive/temp/unlearning/PEM_composition_text/", outputs_dir)
    )

    assert os.path.isdir(os.path.join("/content/Unlearn-Saliency/PEM_composition_text/", trained_adapter_negated_dir))
    shutil.copytree(
        os.path.join("/content/Unlearn-Saliency/PEM_composition_text/", trained_adapter_negated_dir),
        os.path.join("/content/gdrive/MyDrive/temp/unlearning/PEM_composition_text/", trained_adapter_negated_dir)
    )